In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('default_of_credit_card_clients.xls')

In [5]:
import numpy as np

In [6]:
np.random.seed(seed=24)


In [7]:
random_integers = np.random.randint(low=1,high=5,size=100)


In [8]:
is_equal_to_3 = random_integers == 3


In [9]:
is_equal_to_3[:5]


array([ True, False, False, False, False])

In [10]:
sum(is_equal_to_3)


22

In [11]:
random_integers[is_equal_to_3]


array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [13]:
id_counts = df['ID'].value_counts()
dupe_mask = id_counts == 2
dupe_mask[0:5]

04c32736-903c    True
693a0664-bde6    True
823267c1-561b    True
94d8b70f-361d    True
91321f74-e4a4    True
Name: ID, dtype: bool

In [14]:
dupe_ids = id_counts.index[dupe_mask]
dupe_ids = list(dupe_ids)
len(dupe_ids)

313

In [15]:
df.loc[df['ID'].isin(dupe_ids[0:3]),:].head(10) 


ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE PAY_1  PAY_2  \
4576   04c32736-903c      90000    2          1         2   29     1     -1   
4676   04c32736-903c          0    0          0         0    0     0      0   
5753   823267c1-561b     330000    2          2         2   30    -2     -2   
5853   823267c1-561b          0    0          0         0    0     0      0   
21274  693a0664-bde6      50000    1          3         2   51     1     -2   
21374  693a0664-bde6          0    0          0         0    0     0      0   

       PAY_3  PAY_4  ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  \
4576      -1     -2  ...          0          0          0       194         0   
4676       0      0  ...          0          0          0         0         0   
5753      -2     -2  ...          0          0          0         0         0   
5853       0      0  ...          0          0          0         0         0   
21274     -2     -2  ...          0          0          0         0         0   
21374      0      0  ...          0          0          0         0         0   

       PAY_AMT3  PAY_AMT4  PAY_AMT5  PAY_AMT6  default payment next month  
4576          0         0         0       217                           0  
4676          0         0         0         0                           0  
5753          0         0         0         0                           1  
5853          0         0         0         0                           0  
21274         0         0         0         0                           1  
21374         0         0         0         0                           0  

[6 rows x 25 columns]

In [16]:
df_zero_mask = df == 0


In [17]:
feature_zero_mask = df_zero_mask.iloc[:,1:].all(axis=1)
sum(feature_zero_mask)


315

In [18]:
df_clean_1 = df.loc[~feature_zero_mask,:].copy()


In [20]:
df_clean_1['ID'].nunique()


29685

In [21]:
valid_pay_1_mask = df_clean_1['PAY_1'] != 'Not available'
valid_pay_1_mask[0:5]

0    True
1    True
2    True
3    True
4    True
Name: PAY_1, dtype: bool

In [22]:
df_clean_2 = df_clean_1.loc[valid_pay_1_mask,:].copy()


In [23]:
df_clean_2['PAY_1'] = df_clean_2['PAY_1'].astype('int64')
df_clean_2[['PAY_1', 'PAY_2']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26664 entries, 0 to 29999
Data columns (total 2 columns):
PAY_1    26664 non-null int64
PAY_2    26664 non-null int64
dtypes: int64(2)
memory usage: 624.9 KB


In [24]:
df_clean_2['EDUCATION'].replace(to_replace=[0, 5, 6], value=4, inplace=True)
df_clean_2['EDUCATION'].value_counts()

2    12458
1     9412
3     4380
4      414
Name: EDUCATION, dtype: int64

In [25]:
df_clean_2['MARRIAGE'].replace(to_replace=0, value=3, inplace=True)
df_clean_2['MARRIAGE'].value_counts()

2    14158
1    12172
3      334
Name: MARRIAGE, dtype: int64

In [26]:
df_clean_2['EDUCATION_CAT'] = 'none'


In [27]:
df_clean_2[['EDUCATION', 'EDUCATION_CAT']].head(10)


EDUCATION EDUCATION_CAT
0          2          none
1          2          none
2          2          none
3          2          none
4          2          none
5          1          none
6          1          none
7          2          none
8          3          none
9          3          none

In [28]:
cat_mapping = {
    1: "graduate school",
    2: "university",
    3: "high school",
    4: "others"
}

In [29]:
df_clean_2['EDUCATION_CAT'] = df_clean_2['EDUCATION'].map(cat_mapping)
df_clean_2[['EDUCATION', 'EDUCATION_CAT']].head(10)

EDUCATION    EDUCATION_CAT
0          2       university
1          2       university
2          2       university
3          2       university
4          2       university
5          1  graduate school
6          1  graduate school
7          2       university
8          3      high school
9          3      high school

In [32]:
edu_ohe = pd.get_dummies(df_clean_2['EDUCATION_CAT'])
edu_ohe.head(10)
df_with_ohe = pd.concat([df_clean_2, edu_ohe], axis=1)
df_with_ohe[['EDUCATION_CAT', 'graduate school',
             'high school', 'university', 'others']].head(10)
df_with_ohe.to_csv('cleaned_data.csv', index=False)
